<a href="https://colab.research.google.com/github/gigDevelopment10/Aws-cli-commands/blob/main/Covid_data_engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import boto3
import pandas as pd
from io import StringIO

In [ ]:
# Python 3
# !pip install boto3

In [ ]:
AWS_ACCESS_KEY = ''
AWS_SECRET_KEY = ''
AWS_REGION = 'us-east-1'
SCHEMA_NAME = 'covid_dataset'
S3_STAGING_DIR = 's3://de-covid-athena-output/output/'
S3_BUCKET_NAME = 'de-covid-athena-output'
S3_OUTPUT_DIRECTORY = 'output'

In [ ]:
athena_client= boto3.client(
    "athena",
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
    region_name=AWS_REGION,
)

In [ ]:
Dict = {}
import time
def download_and_load_query_results(
    client: boto3.client, query_response: Dict
) -> pd.DataFrame:
    while True:
      try:
        client.get_query_results(
              QueryExecutionId=query_response['QueryExecutionId']
          )
        break
      except Exception as err:
        if 'not yet finished' in str(err):
          time.sleep(0.001)
        else:
          raise err
    temp_file_location: str = "athena_query_results.csv"
    print('Ali')
    s3_client = boto3.client(
        "s3",
        aws_access_key_id=AWS_ACCESS_KEY,
        aws_secret_access_key=AWS_SECRET_KEY,
        region_name=AWS_REGION,
    )
    s3_client.download_file(
        S3_BUCKET_NAME,
        f"{S3_OUTPUT_DIRECTORY}/{query_response['QueryExecutionId']}.csv",
        temp_file_location,   
    )
    return pd.read_csv(temp_file_location)

In [ ]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM countrycode",
    QueryExecutionContext={"Database":SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR ,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)
time.sleep(2)

countrycode = download_and_load_query_results(athena_client, response)
countrycode.head()

Ali


,country,alpha-2 code,alpha-3 code,numeric code,latitude,longitude
0,Afghanistan,AF,AFG,4.0,33.0000,65.0
1,Albania,AL,ALB,8.0,41.0000,20.0
2,Algeria,DZ,DZA,12.0,28.0000,3.0
3,American Samoa,AS,ASM,16.0,-14.3333,-170.0
4,Andorra,AD,AND,20.0,42.5000,1.6


In [ ]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM countypopulation",
    QueryExecutionContext={"Database":SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR ,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)
time.sleep(2)

countypopulation = download_and_load_query_results(athena_client, response)
countypopulation.head()

Ali


,id,id2,county,state,population estimate 2018
0,0500000US01001,1001,Autauga,Alabama,55601
1,0500000US01003,1003,Baldwin,Alabama,218022
2,0500000US01005,1005,Barbour,Alabama,24881
3,0500000US01007,1007,Bibb,Alabama,22400
4,0500000US01009,1009,Blount,Alabama,57840


In [ ]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM enigma_jhud",
    QueryExecutionContext={"Database":SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR ,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)
time.sleep(1)

Dict = {}
import time
def download_and_load_query_results(
    client: boto3.client, query_response: Dict
) -> pd.DataFrame:
    temp_file_location: str = "athena_query_results.csv"
    s3_client = boto3.client(
        "s3",
        aws_access_key_id=AWS_ACCESS_KEY,
        aws_secret_access_key=AWS_SECRET_KEY,
        region_name=AWS_REGION,
    )
    s3_client.download_file(
        S3_BUCKET_NAME,
        f"{S3_OUTPUT_DIRECTORY}/{query_response['QueryExecutionId']}.csv",
        temp_file_location,   
    )
    return pd.read_csv(temp_file_location)
time.sleep(2)

enigma_jhud = download_and_load_query_results(athena_client, response)
enigma_jhud.head()

,fips,admin2,province_state,country_region,last_update,latitude,longitude,confirmed,deaths,recovered,active,combined_key,partition_0
0,NaN,NaN,Anhui,China,2020-01-22T17:00:00,31.826,117.226,1.0,NaN,NaN,NaN,"""Anhui",csv
1,NaN,NaN,Beijing,China,2020-01-22T17:00:00,40.182,116.414,14.0,NaN,NaN,NaN,"""Beijing",csv
2,NaN,NaN,Chongqing,China,2020-01-22T17:00:00,30.057,107.874,6.0,NaN,NaN,NaN,"""Chongqing",csv
3,NaN,NaN,Fujian,China,2020-01-22T17:00:00,26.079,117.987,1.0,NaN,NaN,NaN,"""Fujian",csv
4,NaN,NaN,Gansu,China,2020-01-22T17:00:00,36.061,103.834,NaN,NaN,NaN,NaN,"""Gansu",csv


In [ ]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM enigma_nytimes_data_in_usa_us_states",
    QueryExecutionContext={"Database":SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR ,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)
time.sleep(2)

enigma_nytimes_data_in_usa_us_states = download_and_load_query_results(athena_client, response)
enigma_nytimes_data_in_usa_us_states.head()

,date,state,fips,cases,deaths
0,2020-01-21,Washington,53,1,0
1,2020-01-22,Washington,53,1,0
2,2020-01-23,Washington,53,1,0
3,2020-01-24,Illinois,17,1,0
4,2020-01-24,Washington,53,1,0


In [ ]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM state_abv",
    QueryExecutionContext={"Database":SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR ,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)
time.sleep(2)

state_abv = download_and_load_query_results(athena_client, response)
state_abv.head()

,col0,col1
0,State,Abbreviation
1,Alabama,AL
2,Alaska,AK
3,Arizona,AZ
4,Arkansas,AR


In [ ]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM states_daily",
    QueryExecutionContext={"Database":SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR ,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)
time.sleep(2)

states_daily = download_and_load_query_results(athena_client, response)
states_daily.head()

,date,state,positive,probablecases,negative,pending,totaltestresultssource,totaltestresults,hospitalizedcurrently,hospitalizedcumulative,...,dataqualitygrade,deathincrease,hospitalizedincrease,hash,commercialscore,negativeregularscore,negativescore,positivescore,score,grade
0,20210220,UT,366034,NaN,1507875.0,NaN,totalTestsViral,2788882,260.0,14421.0,...,NaN,8.0,39.0,70f3e22ea3d10f99d5f3c09c55ba95fa1b8aaabb,0.0,0.0,0.0,0.0,0.0,NaN
1,20210220,VA,561812,117662.0,NaN,195.0,totalTestEncountersViral,5728208,1594.0,23436.0,...,NaN,99.0,67.0,75d813bab6075e36b3ed1d3bbbfe18f6692e3959,0.0,0.0,0.0,0.0,0.0,NaN
2,20210220,VI,2575,NaN,43564.0,108.0,posNeg,46139,NaN,NaN,...,NaN,0.0,0.0,7ca160663de572688bb23d17943b6f59863f5fd0,0.0,0.0,0.0,0.0,0.0,NaN
3,20210220,VT,14359,411.0,309335.0,NaN,totalTestsViral,1009285,39.0,NaN,...,NaN,3.0,0.0,5156647b94cb2e59c9e4e26be1943e4827a99f13,0.0,0.0,0.0,0.0,0.0,NaN
4,20210220,WA,332904,17485.0,NaN,NaN,totalTestEncountersViral,5048054,608.0,18969.0,...,NaN,19.0,35.0,8150e925fc2fb429eeb347109e52f7b99ba00f17,0.0,0.0,0.0,0.0,0.0,NaN


In [ ]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM us_county",
    QueryExecutionContext={"Database":SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR ,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)
time.sleep(2)

us_county = download_and_load_query_results(athena_client, response)
us_county.head()

,date,county,state,fips,cases,deaths
0,2020-01-21,Snohomish,Washington,53061.0,1.0,0.0
1,2020-01-22,Snohomish,Washington,53061.0,1.0,0.0
2,2020-01-23,Snohomish,Washington,53061.0,1.0,0.0
3,2020-01-24,Cook,Illinois,17031.0,1.0,0.0
4,2020-01-24,Snohomish,Washington,53061.0,1.0,0.0


In [ ]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM us_daily",
    QueryExecutionContext={"Database":SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR ,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)
time.sleep(2)

us_daily = download_and_load_query_results(athena_client, response)
us_daily.head()

,date,states,positive,negative,pending,hospitalizedcurrently,hospitalizedcumulative,inicucurrently,inicucumulative,onventilatorcurrently,...,lastmodified,recovered,total,posneg,deathincrease,hospitalizedincrease,negativeincrease,positiveincrease,totaltestresultsincrease,hash
0,20210307,56,28755524.0,74579770.0,11808.0,40212.0,878613.0,8137.0,45475.0,2801.0,...,2021-03-07T24:00:00Z,NaN,0,0,839,726,130414,41265,1156241,8b26839690cd05c0cef69cb9ed85641a76b5e78e
1,20210306,56,28714259.0,74449356.0,11783.0,41401.0,877887.0,8409.0,45453.0,2811.0,...,2021-03-06T24:00:00Z,NaN,0,0,1674,503,142201,59620,1409138,d0c0482ea549c9d5c04a7c86acb6fc6a8095a592
2,20210305,56,28654639.0,74307155.0,12213.0,42541.0,877384.0,8634.0,45373.0,2889.0,...,2021-03-05T24:00:00Z,NaN,0,0,2221,2781,271917,68787,1744417,a35ea4289cec4bb55c9f29ae04ec0fd5ac4e0222
3,20210304,56,28585852.0,74035238.0,12405.0,44172.0,874603.0,8970.0,45293.0,2973.0,...,2021-03-04T24:00:00Z,NaN,0,0,1743,1530,177957,65487,1590984,a19ad6379a653834cbda3093791ad2c3b9fab5ff
4,20210303,56,28520365.0,73857281.0,11778.0,45462.0,873073.0,9359.0,45214.0,3094.0,...,2021-03-03T24:00:00Z,NaN,0,0,2449,2172,267001,66836,1406795,9e1d2afda1b0ec243060d6f68a7134d011c0cb2a


In [ ]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM us_total_latest",
    QueryExecutionContext={"Database":SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR ,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)
time.sleep(2)

us_total_latest = download_and_load_query_results(athena_client, response)
us_total_latest.head()

,positive,negative,pending,hospitalizedcurrently,hospitalizedcumulative,inicucurrently,inicucumulative,onventilatorcurrently,onventilatorcumulative,recovered,hash,lastmodified,death,hospitalized,total,totaltestresults,posneg,notes
0,1061101,5170081,2775,53793,111955,9486,4192,4712,373,153947,95064ba29ccbc20dbec397033dfe4b1f45137c99,2020-05-01T09:12:31.891Z,57266,111955,6233957,6231182,6231182,"""NOTE: """"total"""""


In [ ]:
response = athena_client.start_query_execution(
    QueryString="SELECT * FROM usa_hospital_beds_geojson_4cca297c",
    QueryExecutionContext={"Database":SCHEMA_NAME},
    ResultConfiguration={
        "OutputLocation": S3_STAGING_DIR ,
        "EncryptionConfiguration": {"EncryptionOption": "SSE_S3"},
    },
)
time.sleep(2)

usa_hospital_beds_geojson_4cca297c = download_and_load_query_results(athena_client, response)
usa_hospital_beds_geojson_4cca297c.head()

,objectid,hospital_name,hospital_type,hq_address,hq_address1,hq_city,hq_state,hq_zip_code,county_name,state_name,...,num_licensed_beds,num_staffed_beds,num_icu_beds,adult_icu_beds,pedi_icu_beds,bed_utilization,avg_ventilator_usage,potential_increase_in_bed_capac,latitude,longtitude
